In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
import pickle
from tensorflow.keras.utils import Sequence
import random


In [3]:
def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.
    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape)

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (x.max()-x.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [4]:
class DataGenerator(Sequence):

    def __init__(self,path,batch_size=128,epsilon=1e-8 ):
        self.path_img = glob.glob(path + "/*/*.jpg")
        self.batch_size = batch_size
        random.shuffle(self.path_img)
#         print(self.path_img)
        self.n = 0
        self.indexes = np.arange(len(self.path_img))
        self.epsilon = epsilon
    def __len__(self):
        return int(len(self.path_img) / self.batch_size)

    def __getitem__(self, index):

        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # Find list of IDs
#         list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X,y = self._generate_Xy(indexes)
        return X, y


    def on_epoch_end(self):

#         self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            random.shuffle(self.path_img)

    def _generate_Xy(self, list_IDs_temp):
        # Initialization
        
        X = []
        y = []
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img = cv2.imread(self.path_img[ID],0)
            img = cv2.resize(img,(128,128))
            # Calculate FFT
            f = np.fft.fft2(img)
            fshift = np.fft.fftshift(f)
            fshift += self.epsilon
            magnitude_spectrum = 20*np.log(np.abs(fshift))
            # Calculate the azimuthally averaged 1D power spectrum
            psd1D = azimuthalAverage(magnitude_spectrum)
            if "real" in self.path_img[ID]:
                y.append(0)
            else:
                y.append(1)
            X.append(psd1D)
        X = np.array(X)
        y = np.array(y)
        return X,y
    def __next__(self):
        if self.n >= self.__len__():
            self.n = 0
        result = self.__getitem__(self.n)
        self.n += 1
        return result

In [3]:
data = []
label = []
dataset = {}
epsilon = 1e-8


In [ ]:
for filename in glob.glob("/hdd/tam/kaggle/train_imgs/df"+"/*.jpg"):

    img = cv2.imread(filename,0)
    img = cv2.resize(img,(128,128))
    # Calculate FFT
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)

    data.append(psd1D)
    label.append(1)

for filename in glob.glob("/hdd/tam/kaggle/train_imgs/real"+"/*.jpg"):

    img = cv2.imread(filename,0)
    img = cv2.resize(img,(128,128))
    # Calculate FFT
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = azimuthalAverage(magnitude_spectrum)

    data.append(psd1D)
    label.append(0)
    
data = np.array(data)
label = np.array(label)


In [25]:
data

array([[248.09915011, 234.82928505, 232.01475898, ...,  92.4572141 ,
         94.06860653,  89.01081487],
       [262.79359058, 247.17609109, 238.94364201, ...,  82.29797627,
         89.51869639,  87.41373662],
       [244.40581921, 246.26001107, 241.32873579, ...,  90.45115039,
         91.30229992,  92.24602054],
       ...,
       [246.49032036, 240.68592435, 230.3211639 , ...,  78.11573816,
         75.77443065,  84.23578219],
       [246.5114191 , 232.6911902 , 227.427142  , ...,  80.15647112,
         79.56838879,  81.83121556],
       [246.60683459, 232.67204516, 226.34382755, ...,  78.17039694,
         80.33475636,  85.00905148]])

In [ ]:
data[5].shape

In [30]:
dataset["data"] = data
dataset["label"] = label

output = open('/hdd/tam/kaggle/train_imgs/dataset_freq.pkl', 'wb')
pickle.dump(dataset, output)
output.close()
print("DATA Saved")

DATA Saved


In [33]:
num = 5
SVM = 0
LR = 0
for z in range(num):
    # read python dict back from the file
#     pkl_file = open('dataset_freq_1000.pkl', 'rb')
    
#     data = pickle.load(pkl_file)
#     pkl_file.close()
    X = dataset["data"]
    y = dataset["label"]
    X.shape
    try:
        print(z)
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

        from sklearn.svm import SVC
        svclassifier = SVC(kernel='linear')
        svclassifier.fit(X_train, y_train)
        print('Accuracy on test set: {:.3f}'.format(svclassifier.score(X_test, y_test)))
               
        
        from sklearn.linear_model import LogisticRegression
        logreg = LogisticRegression(solver='liblinear', max_iter=1000)
        logreg.fit(X_train, y_train)
        print('Accuracy on test set: {:.3f}'.format(logreg.score(X_test, y_test)))
        
        SVM+=svclassifier.score(X_train, y_train)
        LR+=logreg.score(X_test, y_test)
        
    except:
        num-=1
        print(num)
    

print("Average SVM: "+str(SVM/num))
print("Average LR: "+str(LR/num))

0
Accuracy on test set: 0.652
Accuracy on test set: 0.652
1
Accuracy on test set: 0.622
Accuracy on test set: 0.635
2
Accuracy on test set: 0.627
Accuracy on test set: 0.630
3
Accuracy on test set: 0.617
Accuracy on test set: 0.637
4
4
Average SVM: 0.6504283432554088
Average LR: 0.6386469221835076


In [5]:
import keras
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import MaxPooling2D,Conv2D
from keras.optimizers import SGD

Using TensorFlow backend.


In [6]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2
# config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed
set_session(tf.Session(config=config))

In [14]:
data = DataGenerator("/hdd/tam/kaggle/train_imgs")

In [15]:
data_test = DataGenerator("/hdd/tam/kaggle/test_imgs")

In [8]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
# X_train = X_train/np.max(X_train)
# X_test = X_test/np.max(X_train)

In [9]:
model = keras.models.Sequential([
#     keras.layers.Dense(256,input_dim=179,activation = 'relu'),
#     keras.layers.Dense(128,activation = 'relu'),
#     keras.layers.Dense(64,activation = 'relu'),
#     keras.layers.Dense(128,activation = 'relu'),
    keras.layers.Dense(64,input_dim=88,activation = 'relu'),
    keras.layers.Dense(32,activation = 'relu'),
#     keras.layers.Dense(64,activation = 'relu'),
    keras.layers.Dense(8,activation = 'relu'),
    keras.layers.Dense(1,activation = 'sigmoid')
])


In [10]:
model = keras.models.Sequential([
    keras.layers.Dense(1,input_dim=88,activation = 'sigmoid'),
#     keras.layers.Dense(1,activation = 'sigmoid')
])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 89        
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer = "adam", loss = 'binary_crossentropy',metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit_generator(generator = data,steps_per_epoch=data.__len__(),epochs=100,workers=16,shuffle=True,validation_data=data_test,validation_steps=data_test.__len__())



Epoch 1/100
1374/1374 [==============================] - 1198s 872ms/step - loss: 12.8064 - acc: 0.2055 - val_loss: 11.2323 - val_acc: 0.3031
Epoch 2/100
1374/1374 [==============================] - 1185s 863ms/step - loss: 12.8157 - acc: 0.2049 - val_loss: 11.0896 - val_acc: 0.3120
Epoch 3/100
1374/1374 [==============================] - 1190s 866ms/step - loss: 12.8206 - acc: 0.2046 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 4/100
1374/1374 [==============================] - 1180s 859ms/step - loss: 12.8131 - acc: 0.2050 - val_loss: 11.2995 - val_acc: 0.2990
Epoch 5/100
1374/1374 [==============================] - 1107s 806ms/step - loss: 12.8285 - acc: 0.2041 - val_loss: 11.3624 - val_acc: 0.2951
Epoch 6/100
1374/1374 [==============================] - 1095s 797ms/step - loss: 12.8191 - acc: 0.2047 - val_loss: 11.3624 - val_acc: 0.2951
Epoch 7/100
1374/1374 [==============================] - 1115s 812ms/step - loss: 12.7946 - acc: 0.2062 - val_loss: 11.3037 - val_acc: 0.2987
Epoch

In [ ]:
model.evaluate_generator(validation_data=data_test,validation_steps=data_test.__len__(), workers=8)


In [ ]:
model.fit(X_train, y_train, epochs=1000)


Epoch 1/1000
6887/6887 [==============================] - 2s 350us/step - loss: 0.6960 - acc: 0.4915
Epoch 2/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6954 - acc: 0.5038
Epoch 3/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6945 - acc: 0.5060
Epoch 4/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6938 - acc: 0.5092
Epoch 5/1000
6887/6887 [==============================] - 1s 149us/step - loss: 0.6927 - acc: 0.5121
Epoch 6/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6922 - acc: 0.5130
Epoch 7/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6927 - acc: 0.5142
Epoch 8/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6908 - acc: 0.5252
Epoch 9/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6902 - acc: 0.5264
Epoch 10/1000
6887/6887 [==============================] - 1s 146us/step - loss: 0.6897 - a

6887/6887 [==============================] - 1s 144us/step - loss: 0.6732 - acc: 0.5943
Epoch 161/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6720 - acc: 0.6037
Epoch 162/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6722 - acc: 0.6033
Epoch 163/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6721 - acc: 0.5984
Epoch 164/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6715 - acc: 0.6058
Epoch 165/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6719 - acc: 0.6075
Epoch 166/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6724 - acc: 0.6046
Epoch 167/1000
6887/6887 [==============================] - 1s 146us/step - loss: 0.6710 - acc: 0.6058
Epoch 168/1000
6887/6887 [==============================] - 1s 146us/step - loss: 0.6721 - acc: 0.6023
Epoch 169/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6729

6887/6887 [==============================] - 1s 145us/step - loss: 0.6681 - acc: 0.6090
Epoch 319/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6680 - acc: 0.6075
Epoch 320/1000
6887/6887 [==============================] - 1s 146us/step - loss: 0.6679 - acc: 0.6122
Epoch 321/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6665 - acc: 0.6120
Epoch 322/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6670 - acc: 0.6126
Epoch 323/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6674 - acc: 0.6106
Epoch 324/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6667 - acc: 0.6158
Epoch 325/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6668 - acc: 0.6056
Epoch 326/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6671 - acc: 0.6123
Epoch 327/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6676

6887/6887 [==============================] - 1s 147us/step - loss: 0.6636 - acc: 0.6191
Epoch 477/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6648 - acc: 0.6152
Epoch 478/1000
6887/6887 [==============================] - 1s 142us/step - loss: 0.6650 - acc: 0.6138
Epoch 479/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6643 - acc: 0.6188
Epoch 480/1000
6887/6887 [==============================] - 1s 142us/step - loss: 0.6643 - acc: 0.6167
Epoch 481/1000
6887/6887 [==============================] - 1s 144us/step - loss: 0.6640 - acc: 0.6152
Epoch 482/1000
6887/6887 [==============================] - 1s 143us/step - loss: 0.6653 - acc: 0.6136
Epoch 483/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6640 - acc: 0.6143
Epoch 484/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6640 - acc: 0.6164
Epoch 485/1000
6887/6887 [==============================] - 1s 149us/step - loss: 0.6631

6887/6887 [==============================] - 1s 148us/step - loss: 0.6621 - acc: 0.6180
Epoch 635/1000
6887/6887 [==============================] - 1s 147us/step - loss: 0.6618 - acc: 0.6181
Epoch 636/1000
6887/6887 [==============================] - 1s 145us/step - loss: 0.6621 - acc: 0.6152
Epoch 637/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6626 - acc: 0.6178
Epoch 638/1000
6887/6887 [==============================] - 1s 149us/step - loss: 0.6619 - acc: 0.6172
Epoch 639/1000
6887/6887 [==============================] - 1s 148us/step - loss: 0.6628 - acc: 0.6149
Epoch 640/1000
6887/6887 [==============================] - 1s 149us/step - loss: 0.6626 - acc: 0.6164
Epoch 641/1000
6887/6887 [==============================] - 1s 149us/step - loss: 0.6615 - acc: 0.6180
Epoch 642/1000
6887/6887 [==============================] - 1s 143us/step - loss: 0.6615 - acc: 0.6193
Epoch 643/1000
6887/6887 [==============================] - 1s 146us/step - loss: 0.6626